NAMES: WARA LAURA 
       SUNNY KALSI
COURSE: CS329 STATISTICS AND MACHINE LEARNING 

The following model rates movies based on what people commented about on twitter. 
It works based on a collection of data obtained from twitter developers, cleans the data and detects de level of polarity and subjectivity. 
Since this is an unsupervised method we do not have a training and test data. Instead we have tested the algorithm in two tests of data, and compared it with a popular rating website. 



In [1]:
import pandas as pd
import numpy as np

#data = pd.read_csv('thelastKingdom.csv')
#data = pd.read_csv('tigerKing.csv')
#data = pd.read_csv('extraction_result.csv')
#data = pd.read_csv('upload.csv')
data = pd.read_csv('Ozark.csv')
data


,DATE,ACCOUNT,Transcript
0,2020-05-06 23:56:50,Akeila_small,b'I love Jonah! #Ozark'
1,2020-05-06 23:56:01,TVandDinners,b'I just yelled \xe2\x80\x9ckiss him on the mo...
2,2020-05-06 23:55:49,Sd5182957013,b'@Rogers22L @CapiLady Ruth is my favorite! #O...
3,2020-05-06 23:53:11,Miss_Cati,b'Love #Ozark and heard the S3 ending was craz...
4,2020-05-06 23:51:29,reallyIceAgain,b'Darlene is bay shit crazy #Ozark'
...,...,...,...
995,2020-05-04 17:34:37,OtisOshow,"b""This weekend we went to find the #Ozark main..."
996,2020-05-04 17:23:29,emleschh,b'has Jason Bateman always been this hot or am...
997,2020-05-04 17:12:21,shona__ann,b'Just finished #Ozark and I have no fucking c...
998,2020-05-04 17:04:53,Daily_Express,b'Why is #Ozark so blue? #Ozarkseason3 #OzarkS...


We dropp all the empty rows that did not contain any useful information.

In [2]:
data=data.dropna()
data

,DATE,ACCOUNT,Transcript
0,2020-05-06 23:56:50,Akeila_small,b'I love Jonah! #Ozark'
1,2020-05-06 23:56:01,TVandDinners,b'I just yelled \xe2\x80\x9ckiss him on the mo...
2,2020-05-06 23:55:49,Sd5182957013,b'@Rogers22L @CapiLady Ruth is my favorite! #O...
3,2020-05-06 23:53:11,Miss_Cati,b'Love #Ozark and heard the S3 ending was craz...
4,2020-05-06 23:51:29,reallyIceAgain,b'Darlene is bay shit crazy #Ozark'
...,...,...,...
995,2020-05-04 17:34:37,OtisOshow,"b""This weekend we went to find the #Ozark main..."
996,2020-05-04 17:23:29,emleschh,b'has Jason Bateman always been this hot or am...
997,2020-05-04 17:12:21,shona__ann,b'Just finished #Ozark and I have no fucking c...
998,2020-05-04 17:04:53,Daily_Express,b'Why is #Ozark so blue? #Ozarkseason3 #OzarkS...


We proceed to clean the data. 
We clean the data by elminating unwanted information, and putting all the text in the same format.

In [3]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = text[1:]
    return text

round1 = lambda x: clean_text_round1(x)

In [4]:
data_clean = pd.DataFrame(data.Transcript.apply(round1))
data_clean

,Transcript
0,i love jonah ozark
1,i just yelled him on the to my empty living ...
2,capilady ruth is my favorite ozark
3,love ozark and heard the ending was crazy tbh...
4,darlene is bay shit crazy ozark
...,...
995,this weekend we went to find the ozark main ho...
996,has jason bateman always been this hot or am i...
997,just finished ozark and i have no fucking clue...
998,why is ozark so blue ozark wendy


In [5]:
data_clean.to_pickle("extraction_corpus.pkl")

To keep cleaning the data we apply the split() function to the dataser and store the count of the most common words 

In [6]:
from collections import Counter
common = Counter(" ".join(data_clean["Transcript"]).split()).most_common(20)

In [7]:
df1 = pd.DataFrame(common)
words = df1.rename(columns={0:'Word',1:'count'})
words

,Word,count
0,ozark,1035
1,the,496
2,of,325
3,i,321
4,is,318
5,to,287
6,a,275
7,and,266
8,season,219
9,in,167


In [8]:
import re
import string

pat = '|'.join([r'\b{}\b'.format(w) for w in words['Word']])

Once we have the data together into a string we apply the nltk.stem wich 
will put every word into its base word. 
Following we clean the data one more time removing the 100 most commonly used words.

In [9]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.stem import PorterStemmer, WordNetLemmatizer
porter_stemmer = PorterStemmer()
'''
Write a function to perform the pre processing steps on the entire dataset
'''
stemmer = SnowballStemmer("english")
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

def clean_text_round2(text):
    '''remove 100 most commonly used words'''
    text = re.sub(pat, '', text)
    return text



round2 = lambda x: clean_text_round2(x)

we apply the clean_text_round2 function which removes the most repeated words. We also apply stem_sentences function which puts everyword to its base. 

In [10]:
data_clean['trimmed transcript'] = pd.DataFrame(data_clean.Transcript.apply(round2))
data_clean['stemmed'] = data_clean['trimmed transcript'].apply(stem_sentences)
data_clean


,Transcript,trimmed transcript,stemmed
0,i love jonah ozark,love jonah,love jonah
1,i just yelled him on the to my empty living ...,yelled him empty living room because ...,yell him empti live room becaus am who am
2,capilady ruth is my favorite ozark,capilady ruth favorite,capiladi ruth favorit
3,love ozark and heard the ending was crazy tbh...,love heard ending crazy tbh not crazy ja...,love heard end crazi tbh not crazi jake predic...
4,darlene is bay shit crazy ozark,darlene bay shit crazy,darlen bay shit crazi
...,...,...,...
995,this weekend we went to find the ozark main ho...,weekend we went find main home byrds fro...,weekend we went find main home byrd from netfl...
996,has jason bateman always been this hot or am i...,has jason bateman always been hot or am att...,ha jason bateman alway been hot or am attract ...
997,just finished ozark and i have no fucking clue...,finished have no fucking clue what going ...,finish have no fuck clue what go through whole...
998,why is ozark so blue ozark wendy,why blue wendy,whi blue wendi


We now find the polarity and subjectivity of the already cleaned data using the textblob library and add the columns to the dataframe 

In [11]:
# Create quick lambda functions to find the polarity and subjectivity of each routine
# Terminal / Anaconda Navigator: conda install -c conda-forge textblob
from textblob import TextBlob

pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity

data_clean['polarity'] = data_clean['Transcript'].apply(pol)
data_clean['stemmed_polarity'] = data_clean['stemmed'].apply(pol)
data_clean

,Transcript,trimmed transcript,stemmed,polarity,stemmed_polarity
0,i love jonah ozark,love jonah,love jonah,0.500000,0.500000
1,i just yelled him on the to my empty living ...,yelled him empty living room because ...,yell him empti live room becaus am who am,-0.100000,0.136364
2,capilady ruth is my favorite ozark,capilady ruth favorite,capiladi ruth favorit,0.500000,0.000000
3,love ozark and heard the ending was crazy tbh...,love heard ending crazy tbh not crazy ja...,love heard end crazi tbh not crazi jake predic...,-0.025000,0.262500
4,darlene is bay shit crazy ozark,darlene bay shit crazy,darlen bay shit crazi,-0.400000,-0.200000
...,...,...,...,...,...
995,this weekend we went to find the ozark main ho...,weekend we went find main home byrds fro...,weekend we went find main home byrd from netfl...,0.013889,0.020833
996,has jason bateman always been this hot or am i...,has jason bateman always been hot or am att...,ha jason bateman alway been hot or am attract ...,0.250000,0.250000
997,just finished ozark and i have no fucking clue...,finished have no fucking clue what going ...,finish have no fuck clue what go through whole...,0.250000,0.200000
998,why is ozark so blue ozark wendy,why blue wendy,whi blue wendi,0.000000,0.000000


Now we help the classfication adding 0.2 to base polarity and steemed polarity to not have any values out of range. 
We add two more columns in this section; Predicted Stars and Steemed Predicted Stars, which shows the star rating according to the polarity of the data. 

In [12]:
#add 0.2 to base polarity and stemmed polarity to help with positive words not being classified right
data_clean.polarity = data_clean.polarity+.2
data_clean.stemmed_polarity = data_clean.stemmed_polarity +0.2
data_clean['predicted_stars'] = 0
data_clean['stemmed_predicted_stars'] = 0
data_clean.loc[(data_clean.polarity <-0.8),'predicted_stars']=1
data_clean.loc[(data_clean.stemmed_polarity <-0.8 ),'stemmed_predicted_stars']=1
data_clean.loc[(data_clean.polarity >= -0.8) & (data_clean.polarity < -0.6),'predicted_stars']=2
data_clean.loc[(data_clean.stemmed_polarity >= -0.8) & (data_clean.stemmed_polarity <-0.6 ),'stemmed_predicted_stars']=2
data_clean.loc[(data_clean.polarity >= -0.6) & (data_clean.polarity < -0.4),'predicted_stars']=3
data_clean.loc[(data_clean.stemmed_polarity >= -0.6) & (data_clean.stemmed_polarity <-0.4 ),'stemmed_predicted_stars']=3
data_clean.loc[(data_clean.polarity >= -0.4) & (data_clean.polarity < -0.2),'predicted_stars']=4
data_clean.loc[(data_clean.stemmed_polarity >= 0.4) & (data_clean.stemmed_polarity < -0.2 ),'stemmed_predicted_stars']=4
data_clean.loc[(data_clean.polarity >= -0.2) & (data_clean.polarity < 0),'predicted_stars']=5
data_clean.loc[(data_clean.stemmed_polarity >= -0.2) & (data_clean.stemmed_polarity <0 ),'stemmed_predicted_stars']=5
data_clean.loc[(data_clean.polarity >= 0) & (data_clean.polarity < 0.2),'predicted_stars']=6
data_clean.loc[(data_clean.stemmed_polarity >= 0) & (data_clean.stemmed_polarity <0.2 ),'stemmed_predicted_stars']=6
data_clean.loc[(data_clean.polarity >= 0.2) & (data_clean.polarity < 0.4),'predicted_stars']=7
data_clean.loc[(data_clean.stemmed_polarity >= 0.2) & (data_clean.stemmed_polarity <0.4 ),'stemmed_predicted_stars']=7
data_clean.loc[(data_clean.polarity >= 0.4) & (data_clean.polarity < 0.6),'predicted_stars']=8
data_clean.loc[(data_clean.stemmed_polarity >= 0.4) & (data_clean.stemmed_polarity <0.6),'stemmed_predicted_stars']=8
data_clean.loc[(data_clean.polarity >= 0.6) & (data_clean.polarity < 0.8),'predicted_stars']=9
data_clean.loc[(data_clean.stemmed_polarity >= 0.6) & (data_clean.stemmed_polarity <0.8 ),'stemmed_predicted_stars']=9
data_clean.loc[(data_clean.polarity >= 0.8),'predicted_stars']=10
data_clean.loc[(data_clean.stemmed_polarity >= 0.8),'stemmed_predicted_stars']=10

In [13]:
data_clean

,Transcript,trimmed transcript,stemmed,polarity,stemmed_polarity,predicted_stars,stemmed_predicted_stars
0,i love jonah ozark,love jonah,love jonah,0.700000,0.700000,9,9
1,i just yelled him on the to my empty living ...,yelled him empty living room because ...,yell him empti live room becaus am who am,0.100000,0.336364,6,7
2,capilady ruth is my favorite ozark,capilady ruth favorite,capiladi ruth favorit,0.700000,0.200000,9,7
3,love ozark and heard the ending was crazy tbh...,love heard ending crazy tbh not crazy ja...,love heard end crazi tbh not crazi jake predic...,0.175000,0.462500,6,8
4,darlene is bay shit crazy ozark,darlene bay shit crazy,darlen bay shit crazi,-0.200000,0.000000,5,6
...,...,...,...,...,...,...,...
995,this weekend we went to find the ozark main ho...,weekend we went find main home byrds fro...,weekend we went find main home byrd from netfl...,0.213889,0.220833,7,7
996,has jason bateman always been this hot or am i...,has jason bateman always been hot or am att...,ha jason bateman alway been hot or am attract ...,0.450000,0.450000,8,8
997,just finished ozark and i have no fucking clue...,finished have no fucking clue what going ...,finish have no fuck clue what go through whole...,0.450000,0.400000,8,8
998,why is ozark so blue ozark wendy,why blue wendy,whi blue wendi,0.200000,0.200000,7,7


To see the results we see the mean of every column in our data set using mean() fuction. 

In [14]:
result = data_clean.mean(axis=0)

In [15]:
result

polarity                   0.279714
stemmed_polarity           0.283104
predicted_stars            7.102000
stemmed_predicted_stars    7.066000
dtype: float64

In the following steps we will run three more different datasets from other movies. We will also create a csv file to store the results of every run of each movie

In [16]:
import csv
with open('ratings.csv', 'a+', newline='') as file:
    writer = csv.writer(file)
  #  writer.writerow([Polarity", "Steammed Polarity ", "predicted stars", "stemmed Predicted stars"])
    writer.writerow(result)

After we store the results of different movies run throught the algorithm, we also hand-write the ratings from another website to compare our results. We will display our results.

In [17]:
ratingScore = pd.read_csv('ratings.csv')
ratingScore

,Show/ Movie,Polarity,Steammed Polarity,predicted stars,stemmed Predicted stars,Rooten tomatoe rating
0,1.000000,0.308988,0.285046,7.274151,7.156658,0.91
1,2.000000,0.273936,0.269426,7.082800,7.038700,0.86
2,3.000000,0.370965,0.356915,7.522300,7.463100,0.68
3,4.000000,0.400423,0.371288,7.661240,7.531622,0.83
4,5.000000,0.261314,0.273632,7.021918,7.043836,0.81
5,0.279714,0.283104,7.102000,7.066000,NaN,NaN


At the end of this project we realized that 


1) people are generally a bit more possitive when leaving reviews of movies they did not hate meaning that if they generally thought the movie was okay they would rate it between 3-5, and people generally do not leave a 1 star review unless they absolutly hated the move.

2) the language used in tweets is a bit harder for sentiment analysis to accuratly predict. The difference being that sometimes they talk negativly about a character in the show because of the actions that the character did, but the overall thought on the show could be positive but the rating predicted would be innacurate because we can't tell if they're talking about the show itself or just a character protraied in the show.

3) The sentiment analysis was pretty accurate at detecting a negative vs a positive tweet regardless of the topic. Meaning we can use this method to analyize other data that we can collect from twitter data. 

So how usefull is this tool? I would say that given how much raw data is avaiable on twitter that this is a very powerful tool that can tell you if something (a product) is being portraied negativly or positivily online. With sentiment analysis we can figure out the following.

1) What are people talking about the most (most commonly used words)

2) What people hate about the product (polarity of -1 to -0.5)

3) What people love about the product (polarity of 0.5 to 1)

4) what people think that the product can improve on (-0.2 to 0.2 polarity)
